What is Vaex?

Vaex is a high performance Python library for lazy Out-of-Core DataFrames (similar to Pandas), to visualize and explore big tabular datasets. It calculates statistics such as mean, sum, count, standard deviation etc, on an N-dimensional grid for more than a billion (10^9) samples/rows per second. Visualization is done using histograms, density plots and 3d volume rendering, allowing interactive exploration of big data. Vaex uses memory mapping, zero memory copy policy and lazy computations for best performance (no memory wasted).

model train with large dataset using VAEX

In [1]:

import pandas as pd

In [2]:
df1 = pd.read_csv('large1.csv') # around 3 GB # time for my machine > 2.5 minutes

KeyboardInterrupt: 

In [5]:
%time df = pd.read_csv('large2.csv') # 1-2 GB

<timed exec>:1: DtypeWarning: Columns (28,42,78,82,89) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 14 s
Wall time: 15.4 s


In [7]:
import vaex

ModuleNotFoundError: No module named 'vaex'

Failed to build vaex-core
ERROR: Failed to build installable wheels for some pyproject.toml based projects (vaex-core)

example 1

Create Hdf5 files

In [ ]:
file_path = 'large2.csv'

In [ ]:
vaex_df = vaex.from_csv(file_path, convert=True, chunk_size=5_000_000)

In [ ]:
# vaex_df = vaex.from_pandas(df)

In [ ]:
# %time vaex_df = vaex.open("large_data.hdf5")

In [ ]:
type(vaex_df)

Read Hdf5 files using Vaex library

In [ ]:
vaex_df = vaex.open('large2.csv.hdf5')

In [ ]:
type(vaex_df)

In [ ]:
print("Size =") 
print(df.shape) 
print(vaex_df.shape) 

In [ ]:
vaex_df.head()

In [ ]:
vaex_df = vaex_df.shuffle()
df_train, df_test = vaex_df.ml.train_test_split(test_size=0.2)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor

In [ ]:
features = []
target = ''


In [ ]:
model = SGDRegressor()

vaex_model = IncrementalPredictor(features=features,target=target,model=model,batch_size=500000)

vaex_model.fit(df=df_train,progress='widget')

In [ ]:
df_test = vaex_model.transform(df_test)

In [ ]:
df_test.head()

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(df_test['target'].values,df_test['prediction'].values))
print(mean_absolute_error(df_test['target'].values,df_test['prediction'].values))

2nd example

In [ ]:
import vaex
import pandas as pd
import numpy as np
n_rows = 1000000
n_cols = 500
df = pd.DataFrame(np.random.randint(0, 100, size=(n_rows, n_cols)), columns=['col%d' % i for i in range(n_cols)])
df.head()

In [ ]:
df.info(memory_usage='deep')

Creating Csv files

In [ ]:
file_path = 'final_data.csv'
df.to_csv(file_path, index=False)

Create Hdf5 files

In [ ]:
vaex_df = vaex.from_csv(file_path, convert=True, chunk_size=5_000_000)

In [ ]:
type(vaex_df)

Read Hdf5 files using Vaex library

In [ ]:
vaex_df = vaex.open('final_data.csv.hdf5')

In [ ]:
type(vaex_df)

In [ ]:
vaex_df.head()

Expression system

Don't waste memory or time with feature engineering, we (lazily) transform your data when needed.

In [ ]:
%%time
vaex_df['multiplication_col13']=vaex_df.col1*vaex_df.col3

In [ ]:
vaex_df['multiplication_col13']

Out-of-core DataFrame

Filtering and evaluating expressions will not waste memory by making copies; the data is kept untouched on disk, and will be streamed only when needed. Delay the time before you need a cluster.

In [ ]:
vaex_df[vaex_df.col2>70]

In [ ]:

dff=vaex_df[vaex_df.col2>70]  ##Here Filtering will not make a memory c

In [ ]:
### All the agorithms work out of core, the limit is the size of your harddriver
dff.col2.minmax(progress='widget')

Fast groupby / aggregations

Vaex implements parallelized, highly performant groupby operations, especially when using categories (>1 billion/second).

In [ ]:
%%time
vaex_df_group=vaex_df.groupby(vaex_df.col1,agg=vaex.agg.mean(vaex_df.col4))
vaex_df_group

In [ ]:
%%time
vaex_df.groupby(vaex_df.col1,agg='count')